In [2]:
import praw

# Authenticate with Reddit
reddit = praw.Reddit(client_id='uDNAhJjTINgI5xsJCa33OQ',
                     client_secret='ZUonzednDbeNVz0khjdJowfmbMufCg',
                     user_agent='WebScraping by /u/happybirdie007',
                     username='happybirdie007',
                     password='Ai1Jiao2Er?')

# Define keywords and subreddit
keywords = ['gpt store', 'gpt plugin']  # Add your keywords here
subreddit = reddit.subreddit('all')  # Change to a specific subreddit if needed

# Search and process posts
for keyword in keywords:
    for submission in subreddit.search(keyword, sort='new'):
        print(f"Title: {submission.title}")
        print(f"URL: {submission.url}\n")
        break


Title: AI app on your own website potentially makes more than GPT store
URL: https://www.youtube.com/watch?v=mVmDROhUA6A&t=22s

Title: Can Zapier interact with custom made GPT?
URL: https://www.reddit.com/r/ChatGPT/comments/19fbv6c/can_zapier_interact_with_custom_made_gpt/



In [4]:
from selenium.webdriver.common.by import By
# import undetected_chromedriver as uc
from selenium import webdriver
import pandas as pd
import requests

tag_store_links = []
tag_plugin_links = []

driver = webdriver.Chrome()
driver.implicitly_wait(5)

url = 'https://community.openai.com/tags'
driver.get(url)

for tag in driver.find_elements(By.XPATH, '//div[@class="tag-box"]/a'):
    tag_name = tag.get_attribute('data-tag-name')
    if 'store' in tag_name:
        tag_store_links.append(tag.get_attribute('href'))
    elif 'plugin' in tag_name:
        tag_plugin_links.append(tag.get_attribute('href'))
        
post_store_links = set()
post_plugin_links = set()

for tag_link in tag_store_links:
    driver.get(tag_link)
    for post in driver.find_elements(By.XPATH, '//tbody[@class="topic-list-body"]/tr'):
        post_link = post.find_element(By.XPATH, './/a[@role="heading"]').get_attribute('href')
        post_store_links.add(post_link)

for tag_link in tag_plugin_links:
    driver.get(tag_link)
    for post in driver.find_elements(By.XPATH, '//tbody[@class="topic-list-body"]/tr'):
        post_link = post.find_element(By.XPATH, './/a[@role="heading"]').get_attribute('href')
        post_plugin_links.add(post_link)
        
posts = pd.DataFrame()
for post_link in post_store_links:
    post_link = post_link + '.json'
    json_data = requests.get(post_link).json()
    post = {}
    post['Title'] = json_data['title']
    post['Creation Time'] = json_data['created_at']
    post['View Count'] = json_data['views']
    post['Reply Count'] = json_data['reply_count']
    post['Like Count'] = json_data['like_count']
    post['Link'] = post_link
    post['Question'] = json_data['post_stream']['posts'][0]['cooked']
    post['Replies'] = '\n'.join([reply['cooked'] for reply in json_data['post_stream']['posts'][1:]])
    post['Tags'] = json_data['tags']
    post = pd.DataFrame([post])
    posts = pd.concat([posts, post], ignore_index=True)
posts.to_json('openai_store_posts.json', orient='records', indent=4)
        
posts = pd.DataFrame()
for post_link in post_store_links:
    post_link = post_link + '.json'
    json_data = requests.get(post_link).json()
    post = {}
    post['Title'] = json_data['title']
    post['Creation Time'] = json_data['created_at']
    post['View Count'] = json_data['views']
    post['Reply Count'] = json_data['reply_count']
    post['Like Count'] = json_data['like_count']
    post['Link'] = post_link
    post['Question'] = json_data['post_stream']['posts'][0]['cooked']
    post['Replies'] = '\n'.join([reply['cooked'] for reply in json_data['post_stream']['posts'][1:]])
    post['Tags'] = json_data['tags']
    post = pd.DataFrame([post])
    posts = pd.concat([posts, post], ignore_index=True)
posts.to_json('openai_plugin_posts.json', orient='records', indent=4)
    